In [7]:
import os
from PIL import Image
import json
from torchvision import transforms
import torch
from sklearn.metrics import precision_recall_fscore_support, confusion_matrix
from tqdm import tqdm
from transformers import ViTFeatureExtractor, ViTForImageClassification
from functools import partial
import shutil
import time


def write_to_txt(filepath, content):
    with open(filepath, "w") as file:
        file.write(content)

def load_images_from_directory(root_path: str):
    dataset = []
    for label in os.listdir(root_path):
        label_path = os.path.join(root_path, label)
        if os.path.isdir(label_path):
            for image_file in os.listdir(label_path):
                image_path = os.path.join(label_path, image_file)
                if image_path.lower().endswith(('.png', '.jpg', '.jpeg')):
                    img = Image.open(image_path)
                    dataset.append((img, label, image_file))
    return dataset

current_dir = "/home/workstation/code/XAImethods/CAIN"
dataset_path = f"{current_dir}/imagenet/val_images10k_attack/pixelate/5"
dataset = load_images_from_directory(dataset_path)

# Load ImageNet class index
with open(f"{current_dir}/imagenet/imagenet_class_index.json", "r") as f:
    imagenet_class_index = json.load(f)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

label_to_index_description = {v[0]: (k, v[1]) for k, v in imagenet_class_index.items()}

# Initialize the ViT model
model = ViTForImageClassification.from_pretrained('google/vit-large-patch32-384').to(device)
model.eval()

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

transform = transforms.Compose([
    transforms.Resize((384, 384)), # Adjust size for ViT
    transforms.ToTensor(),
    normalize
])

def ensure_rgb(img):
    if img.mode != 'RGB':
        return img.convert('RGB')
    return img

true_labels = []
predicted_labels = []

# Create the target directory if it does not exist
target_dir = dataset_path + "_vit"
os.makedirs(target_dir, exist_ok=True)
start_time = time.time()
for img, label, filename in tqdm(dataset):
    img = ensure_rgb(img)  
    img_tensor = transform(img).to(device)
    
    # Model prediction
    with torch.no_grad():
        logits = model(img_tensor.unsqueeze(0)).logits
        predicted_label = torch.argmax(logits, dim=1).item()
    
    # Get true label
    index_str, _ = label_to_index_description.get(label, (None, None))
    if index_str is None:
        continue
    true_label = int(index_str)
    true_labels.append(true_label)
    predicted_labels.append(predicted_label)
    # If prediction is correct, copy the image to the target directory
    if true_label == predicted_label:
        source_path = os.path.join(dataset_path, label, filename)
        target_label_dir = os.path.join(target_dir, label)
        os.makedirs(target_label_dir, exist_ok=True)  # Make sure the directory exists
        shutil.copy(source_path, target_label_dir)  # Copy the file

precision, recall, f1, _ = precision_recall_fscore_support(true_labels, predicted_labels, average='macro')
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
end_time = time.time()
prediction_time = end_time - start_time
print(f"Total Prediction Time: {prediction_time:.2f} seconds")


results = []

results.append(f"Precision: {precision:.4f}\n")
results.append(f"Recall: {recall:.4f}\n")
results.append(f"F1 Score: {f1:.4f}\n")
results.append(f"Total Prediction Time: {prediction_time:.2f} seconds\n")
results.append("\n")

for target_class in set(true_labels):
    # Use sklearn's confusion matrix
    matrix = confusion_matrix(true_labels, predicted_labels, labels=[target_class])
    if matrix.size == 1:
        value = matrix[0][0]
        if target_class in predicted_labels:  # It was predicted at least once
            tp = value
            tn, fp, fn = 0, 0, 0
        else:  # It was never predicted
            tn = value
            tp, fp, fn = 0, 0, 0
    else:
        tn, fp, fn, tp = matrix.ravel()
    results.append(f"For class {target_class}: TP={tp}, FP={fp}, FN={fn}, TN={tn}\n")
    print(f"For class {target_class}: TP={tp}, FP={fp}, FN={fn}, TN={tn}")



results_path = os.path.join(target_dir, "results.txt")
write_to_txt(results_path, ''.join(results))



100%|██████████| 10000/10000 [03:10<00:00, 52.44it/s]
/home/workstation/anaconda3/envs/xai/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Precision: 0.5882
Recall: 0.4991
F1 Score: 0.5001
Total Prediction Time: 190.70 seconds
For class 0: TP=6, FP=0, FN=0, TN=0
For class 1: TP=8, FP=0, FN=0, TN=0
For class 2: TP=7, FP=0, FN=0, TN=0
For class 3: TP=7, FP=0, FN=0, TN=0
For class 4: TP=7, FP=0, FN=0, TN=0
For class 5: TP=6, FP=0, FN=0, TN=0
For class 6: TP=3, FP=0, FN=0, TN=0
For class 7: TP=8, FP=0, FN=0, TN=0
For class 8: TP=8, FP=0, FN=0, TN=0
For class 9: TP=6, FP=0, FN=0, TN=0
For class 10: TP=6, FP=0, FN=0, TN=0
For class 11: TP=6, FP=0, FN=0, TN=0
For class 12: TP=8, FP=0, FN=0, TN=0
For class 13: TP=6, FP=0, FN=0, TN=0
For class 14: TP=8, FP=0, FN=0, TN=0
For class 15: TP=8, FP=0, FN=0, TN=0
For class 16: TP=8, FP=0, FN=0, TN=0
For class 17: TP=10, FP=0, FN=0, TN=0
For class 18: TP=6, FP=0, FN=0, TN=0
For class 19: TP=8, FP=0, FN=0, TN=0
For class 20: TP=3, FP=0, FN=0, TN=0
For class 21: TP=4, FP=0, FN=0, TN=0
For class 22: TP=10, FP=0, FN=0, TN=0
For class 23: TP=9, FP=0, FN=0, TN=0
For class 24: TP=4, FP=0, FN=0, 